In [0]:
# Import functions
from pyspark.sql.functions import col, current_timestamp

# Define variables used in code below
file_path = "/FileStore/stream/events/"
username = spark.sql("SELECT regexp_replace(current_user(), '[^a-zA-Z0-9]', '_')").first()[0]
table_name = f"{username}_etl_quickstart"
checkpoint_path = f"/tmp/{username}/_checkpoint/etl_quickstart"

# Clear out data from previous demo execution
spark.sql(f"DROP TABLE IF EXISTS {table_name}")
dbutils.fs.rm(checkpoint_path, True)



Out[1]: False

In [0]:
dbutils.fs.rm(f"dbfs:/user/hive/warehouse/{username}_etl_quickstart", True)


Out[2]: False

In [0]:
%fs ls /databricks-datasets/structured-streaming/events

path,name,size,modificationTime
dbfs:/databricks-datasets/structured-streaming/events/file-0.json,file-0.json,72530,1469673865000
dbfs:/databricks-datasets/structured-streaming/events/file-1.json,file-1.json,72961,1469673866000
dbfs:/databricks-datasets/structured-streaming/events/file-10.json,file-10.json,73025,1469673878000
dbfs:/databricks-datasets/structured-streaming/events/file-11.json,file-11.json,72999,1469673879000
dbfs:/databricks-datasets/structured-streaming/events/file-12.json,file-12.json,72987,1469673880000
dbfs:/databricks-datasets/structured-streaming/events/file-13.json,file-13.json,73006,1469673881000
dbfs:/databricks-datasets/structured-streaming/events/file-14.json,file-14.json,73003,1469673882000
dbfs:/databricks-datasets/structured-streaming/events/file-15.json,file-15.json,73007,1469673883000
dbfs:/databricks-datasets/structured-streaming/events/file-16.json,file-16.json,72978,1469673885000
dbfs:/databricks-datasets/structured-streaming/events/file-17.json,file-17.json,73008,1469673886000


Żeby sprawdzić czy stream działa kopjuj po jednym bądź kilku plikach 

In [0]:
dbutils.fs.cp("/databricks-datasets/structured-streaming/events/file-7.json","/FileStore/stream/events/file-7.json",True)

Out[11]: True

In [0]:
display(dbutils.fs.ls("/FileStore/stream/events/"))

path,name,size,modificationTime
dbfs:/FileStore/stream/events/file-4.json,file-4.json,72998,1746297085000
dbfs:/FileStore/stream/events/file-5.json,file-5.json,72998,1746297093000
dbfs:/FileStore/stream/events/file-6.json,file-6.json,72997,1746297101000
dbfs:/FileStore/stream/events/file-7.json,file-7.json,73022,1746297110000


In [0]:
spark.read.format("json").load("dbfs:/FileStore/stream/events/").count()

Out[5]: 2000

Dokończyć kod autoloadera 
1. Dodaj opcje 'cloudfiles'
2. Dodaj kolumnę z metadanych 'source_file'
3. Dane zapisać do tabeli

In [0]:
from pyspark.sql.functions import input_file_name


(spark.readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "json")  # typ plików (np. json, csv, parquet)
  .option("cloudFiles.schemaLocation", checkpoint_path)  # miejsce trzymania schematu
  .load(file_path)
  .withColumn("source_file", input_file_name())
  .writeStream
  .option("checkpointLocation", checkpoint_path)
  .trigger(availableNow=True)
  .toTable(table_name))


Out[13]: <pyspark.sql.streaming.query.StreamingQuery at 0x7f6b7e612e50>

In [0]:
df = spark.sql(f"select count(*) from {table_name}")
df.display()

count(1)
8000


Sprawdzć metadane 

In [0]:
display(dbutils.fs.ls(f"/tmp/{username}/_checkpoint/etl_quickstart/sources/0/metadata/"))

path,name,size,modificationTime
dbfs:/tmp/sanek_student_agh_edu_pl/_checkpoint/etl_quickstart/sources/0/metadata,metadata,133,1746297219000


In [0]:
%fs ls /tmp/{user}/_checkpoint/etl_quickstart/metadata

at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:121)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:179)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:161)
	at com.databricks.backend.daemon.dbutils.FSUtils.lsWithLimit(DBUtilsCore.scala:274)
	at com.databricks.backend.daemon.dbutils.FSUtils.$anonfun$lsImpl$4(DBUtilsCore.scala:243)
	at com.databricks.backend.daemon.dbutils.FSUtils.withFsSafetyCheck(DBUtilsCore.scala:149)
	at com.databricks.backend.daemon.dbutils.FSUtils.$anonfun$lsImpl$3(DBUtilsCore.scala:243)
	at com.databricks.backend.daemon.dbutils.FSUtils.checkPermission(DBUtilsCore.scala:144)
	at com.databricks.backend.daemon.dbutils.FSUtils.lsImpl(DBUtilsCore.scala:242)
	at com.databricks.backend.daemon.dbutils.FSUtils.$anonfun$ls$1(DBUtilsCore.scala:215)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:657)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:678)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:414)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:158)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:412)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:409)
	at com.databricks.backend.daemon.dbutils.FSUtils.withAttributionContext(DBUtilsCore.scala:71)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:457)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:442)
	at com.databricks.backend.daemon.dbutils.FSUtils.withAttributionTags(DBUtilsCore.scala:71)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:652)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:569)
	at com.databricks.backend.daemon.dbutils.FSUtils.recordOperationWithResultTags(DBUtilsCore.scala:71)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:560)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:528)
	at com.databricks.backend.daemon.dbutils.FSUtils.recordOperation(DBUtilsCore.scala:71)
	at com.databricks.backend.daemon.dbutils.FSUtils.recordDbutilsFsOp(DBUtilsCore.scala:135)
	at com.databricks.backend.daemon.dbutils.FSUtils.ls(DBUtilsCore.scala:215)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.ls(DbfsUtilsImpl.scala:67)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-2658262237014856:1)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-2658262237014856:43)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$$iw$$iw$$iw$$iw.<init>(command-2658262237014856:45)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$$iw$$iw$$iw.<init>(command-2658262237014856:47)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$$iw$$iw.<init>(command-2658262237014856:49)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$$iw.<init>(command-2658262237014856:51)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read.<init>(command-2658262237014856:53)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$.<init>(command-2658262237014856:57)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$read$.<clinit>(command-2658262237014856)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$eval$.$print$lzycompute(<notebook>:7)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$eval$.$print(<notebook>:6)
	at $line213b7d88104b4cb1ace311a0a0583bef29.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.i